In [11]:
from seleniumbase import Driver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException

In [12]:
driver = Driver(uc=True)   # undetected-chromedriver

In [13]:
BASE_URL = 'https://www.noon.com/egypt-ar/search/?page={}&q=%D8%A7%D9%84%D8%A7%D8%AF%D9%88%D8%A7%D8%AA+%D8%A7%D9%84%D8%B5%D8%AD%D9%8A%D9%87+%D9%84%D9%84%D8%AD%D9%85%D8%A7%D9%85+%D8%AE%D9%84%D8%A7%D8%B7%D8%A7%D8%AA&utm_source=C1000088L&utm_medium=cpc&utm_campaign=C1000151355N_eg_en_web_searchxxexactandphrasexxbrandpurexx08082022_noon_web_c1000088l_acquisition_sembranded'

In [14]:
all_product_links = []

In [15]:
for i in range(1,13):
    time.sleep(3) 
    driver.get(BASE_URL.format(i))
    PRODUCT_NAME_LOCATOR = (By.XPATH, '//h2[@data-qa="plp-product-box-name"]')
    product_name_elements = driver.find_elements(*PRODUCT_NAME_LOCATOR)
    
    if not product_name_elements:
        print("No product elements found on the page.")
    
    for index, name_element in enumerate(product_name_elements):
        link_element = name_element.find_element(By.XPATH, './ancestor::a[1]')
        product_link = link_element.get_attribute('href')
        all_product_links.append(product_link)


In [16]:
all_product_links

['https://www.noon.com/egypt-ar/160cm-sink-and-drain-plunger-sink-plunger-kitchen-sink-plunger-suction-plunger-multi-color-toilet-cleaner-with-retractable-claw-wand-160cm-kitchen-and-bathroom-drain-plunger-flexible-snake-shaped-retractable-drain-unblocker-with-claws-for-cleaning-an/Z310CC22D4B4D9ACC9E56Z/p/?o=z310cc22d4b4d9acc9e56z-1',
 'https://www.noon.com/egypt-ar/2/Z50A17AF7F95F36C985E9Z/p/?o=z50a17af7f95f36c985e9z-1',
 'https://www.noon.com/egypt-ar/bathroom-shattaf-with-a-2-meter-hose-for-travel-in-a-small-bag-with-a-2-liter-desert-shattaf-bottle-in-the-event-that-there-is-no-water-source/Z13F5FB2807891EE178C8Z/p/?o=z13f5fb2807891ee178c8z-1',
 'https://www.noon.com/egypt-ar/bathroom-shattaf-with-golden-hose-and-hook-code-202/ZF476AA6D6E051DD848CBZ/p/?o=zf476aa6d6e051dd848cbz-1',
 'https://www.noon.com/egypt-ar/a-complete-bathroom-coupling-machine-that-works-on-most-models-such-as-ideal-durevite-gravina-and-others/Z8DC28244D3E28AEB57CAZ/p/?o=z8dc28244d3e28aeb57caz-1',
 'https://ww

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

def extract_noon_reviews(driver, url, timeout=15):
    """
    Extract all reviews from a Noon product page.
    
    Args:
        driver: Selenium WebDriver
        url: Product URL
        timeout: Max seconds to wait for reviews to load

    Returns:
        List of dictionaries with review info
    """
    driver.get(url)
    all_reviews = []
    wait = WebDriverWait(driver, timeout)
    product_name = driver.find_element(By.CSS_SELECTOR, "[class*='ProductTitle']").text

    # ---- Gradual scroll until reviews section is visible ----
    def scroll_to_reviews():
        for _ in range(5):
            try:
                review_section = driver.find_element(By.XPATH, "//div[contains(@class,'ProductReviewsFilters-module') and contains(., 'المراجعات')]")
                if review_section.is_displayed():
                    driver.execute_script("arguments[0].scrollIntoView(true);", review_section)
                    return True
            except:
                pass
            driver.execute_script("window.scrollBy(0, 550);")
            time.sleep(1.5)
        return False

    if not scroll_to_reviews():
        print("⚠️ Reviews section not found on page")
        return []

    time.sleep(1)  # small wait for rendering

    # ---- Click "View All Reviews" if exists ----
    try:
        view_all = driver.find_element(By.CSS_SELECTOR, ".ViewAllReviewsBtn-module-scss-module__viewMore")
        driver.execute_script("arguments[0].scrollIntoView(true);", view_all)
        time.sleep(0.5)
        view_all.click()
        time.sleep(2)  # wait for all reviews page to load
    except NoSuchElementException:
        pass  # no "View All Reviews" button

    # ---- Wait until at least one review is visible or timeout ----
    try:
        wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, ".ReviewListSection-module-scss-module__JSH-ya__noonReviewItem")
            )
        )
    except TimeoutException:
        print("⚠️ No reviews found for this product")
        return []

    # ---- Extract reviews ----
    review_containers = driver.find_elements(
        By.CSS_SELECTOR, ".ReviewListSection-module-scss-module__JSH-ya__noonReviewItem"
    )

    for r in review_containers:
        try:
            username = r.find_element(By.CSS_SELECTOR, ".ReviewItemHeader-module-scss-module__W2V5NW__userName").text
        except:
            username = "anonymous"

        try:
            date = r.find_element(By.CSS_SELECTOR, ".ReviewItemHeader-module-scss-module__W2V5NW__ratedDate").text
        except:
            date = ""

        try:
            title = r.find_element(By.CSS_SELECTOR, ".TranslatedReviewTitle-module-scss-module__QbaJfW__reviewTitle").text
        except:
            title = ""

        try:
            body = r.find_element(By.CSS_SELECTOR, ".ReviewDescription-module-scss-module__AkZflW__reviewDesc").text
        except:
            body = ""

        try:
            rating_imgs = r.find_elements(By.CSS_SELECTOR, ".RatingStars-module-scss-module__fF13gq__container img")
            rating = sum(1 for img in rating_imgs if "rating" in img.get_attribute("color") and "grey" not in img.get_attribute("color"))
        except:
            rating = None

        all_reviews.append({
            "product": product_name,
            "username": username,
            "date": date,
            "title": title,
            "text": body,
            "rating": rating
        })

    print(f"✅ Found {len(all_reviews)} review(s)")
    return all_reviews


In [18]:
# url = "https://www.noon.com/egypt-ar/loq-gaming-laptop-with-15-6-inch-full-hd-1920x1080-ips-display-core-i5-12450hx-processor-16gb-ddr5-ram-512gb-ssd-4gb-nvidia-geforce-rtx-2050-graphics-dos-without-windows-english-arabic-grey/N70148823V/p/?utm_source=C1000088L&utm_medium=cpc&utm_campaign=C1000151355N_eg_en_web_searchxxexactandphrasexxbrandpurexx08082022_noon_web_c1000088l_acquisition_sembranded&o=caaa680fb6a8d82a&shareId=5ea23957-22eb-40d3-8a33-91ee8940ab29"
# data = extract_noon_reviews(driver,url)

In [19]:
# data

In [ ]:
all_reviews = []
for url in all_product_links:
    reviews = extract_noon_reviews(driver,url)
    all_reviews.extend(reviews)  # Combine all reviews in one list

# Convert to DataFrame



✅ Found 1 review(s)
✅ Found 1 review(s)
✅ Found 2 review(s)
✅ Found 2 review(s)
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on 

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
#0 0x5d9428ddc08a <unknown>
#1 0x5d942887b8c3 <unknown>
#2 0x5d94288beecf <unknown>
#3 0x5d94288f32e6 <unknown>
#4 0x5d94288edde7 <unknown>
#5 0x5d94288ecf25 <unknown>
#6 0x5d9428843bd8 <unknown>
#7 0x5d9428da125b <unknown>
#8 0x5d9428da4fa9 <unknown>
#9 0x5d9428d88339 <unknown>
#10 0x5d9428da5b58 <unknown>
#11 0x5d9428d6cc1f <unknown>
#12 0x5d9428841a24 <unknown>
#13 0x7782db62a1ca <unknown>
#14 0x7782db62a28b __libc_start_main
#15 0x5d9428805eea _start


In [22]:
df_all_reviews = pd.DataFrame(all_reviews)
df_all_reviews.head()

,product,username,date,title,text,rating
0,سلاكة حوض وبلاعات 160 سم - سلاكة بلاعة - سلاكة...,Anonymous,"أكتوبر 28, 2025",ممتاز,ممتاز,5
1,شطاف حمام بخرطوم زجزاج مع خرطوم 2 متر اضافى مع...,Mohamed A.,"أكتوبر 3, 2025",مش زي الوصف,جي من غير المحبس اللي هوه اهم حاجه,1
2,شطاف حمام بخرطوم 2 متر 6 مم للسفر,Anonymous,"نوفمبر 1, 2023",كويس بس خرطوم قصير وبيسرب ميه,كويس,3
3,شطاف حمام بخرطوم 2 متر 6 مم للسفر,Mayyada M.,"نوفمبر 20, 2024",نفس الشكل مع القطع بس مجربتهوش لسة,يخد مياه اكتر,4
4,شطاف حمام نحاس دهبى,Anonymous,"مايو 26, 2024",Very good,Gooooooooood,5


In [23]:
df_all_reviews.duplicated().sum()

np.int64(0)

In [24]:
df_all_reviews.drop_duplicates(inplace=True)

In [25]:
df_all_reviews.shape

(19, 6)

In [26]:
df_all_reviews.to_csv("all_noon_reviews_خلاطات.csv", index=False)